In [1]:
# Excel Data Load
import pandas as pd
file_path = './data/Copyright_case_law.xlsx'
df = pd.read_excel(file_path)
from langchain_community.document_loaders import DataFrameLoader
# DataFrame Loader
loader = DataFrameLoader(df, page_content_column="판례내용")
docs = loader.load()
# Data 확인
print('Length of Data:',len(docs))
print('First page:',docs[0].page_content)
print('First metadata:',docs[0].metadata)

Length of Data: 1492
First page: 【주 문】
 ‘특정범죄 가중처벌 등에 관한 법률’(2010. 3. 31. 법률 제10210호로 개정된 것) 제6조 제7항
 중
 관세법 제271조 제3항
 가운데 제269조 제2항에 관한 부분은 헌법에 위반된다.
 이유
 1. 사건개요
 가. 당해사건의 피고인 정○영은 조○경 등과 사이에, 위조 상품을 정상 상품으로 위장하여 수입하기로 공모하고, 시가 합계 30억 5,670만 원 상당의 위조 상품을 적입한 컨테이너를 인천항에 반입하면서 면봉을 수입하는 것처럼 적하목록을 제출하였으나 수입신고 전에 위 컨테이너가 세관직원들에 의해 적발되어, 해당 수입물품을 다른 물품으로 수입할 목적으로 밀수입을 예비하였고, 피고인 김○성은 위와 같이 밀수입을 목적으로 반입되어 인천세관 장치장에 보관 중이던 위조 상품 중 일부를 반출할 목적으로, 관세무역개발원 소속 직원에게 반출을 요구하였다가 거절당하여, 신고하지 않고 외국물품을 수입할 목적으로 밀수입을 예비하였다는 공소사실로 인천지방법원에 기소되었다(인천지방법원 2015고합237).
 나. 위 법원은 2015. 10. 2. 피고인 정○영에 대해서는
 ‘특정범죄 가중처벌 등에 관한 법률’ 제6조 제7항
 , 제2항 제1호, 제6항 제2호,
 관세법 제271조 제3항
 , 제269조 제2항 제2호
 , 제241조 제1항
 ,
 형법 제30조
 를 적용하여 징역 3년, 집행유예 4년 및 벌금 1,930,906,156원에 처하고 압수품은 몰수한다는 판결을, 피고인 김○성에 대해서는
 ‘특정범죄 가중처벌 등에 관한 법률’ 제6조 제7항
 , 제2항 제1호, 제6항 제2호,
 관세법 제271조 제3항
 , 제269조 제2항 제1호
 , 제241조 제1항
 을 적용하여 징역 2년 6월, 집행유예 3년 및 벌금 1,295,329,648원에 처하는 판결을 선고하였고, 피고인들은 위 판결에 불복하여 서울고등법원에 항소하였다(서울고등법원 2015노2940).
 다. 서울고등법원은 항소심 계

In [ ]:
# Vector Embedding
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv()

# --- 임베딩 model loading: text-embedding-ada-002 ---

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
EMB_MODEL = "text-embedding-ada-002"  # 1536차원. 테이블 차원도 맞추면 좋음.
EMB_DIM = 1536

# --- 데이터 로드 ---
df = pd.read_excel("./data/Copyright_case_law.xlsx")

# 컬럼 매핑
# ['사건명','사건번호','선고일자','법원명','판례내용']
df = df.rename(columns={
    '사건명':'case_title','사건번호':'case_no','선고일자':'decided_at','법원명':'court','판례내용':'content'
})
# 날짜 정규화
def to_date(x):
    try:
        return pd.to_datetime(x).date()
    except:
        return None
df['decided_at'] = df['decided_at'].apply(to_date)

# 긴 content를 여러 조각으로 RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])

#  split 완료된 content column의 chunk 데이터를 rows 배열에 임시 저장. // rows => laws.cases table에 INSERT할 임시 저장소
rows = []
for idx, row in df.iterrows():
    base_text = f"{row.get('case_title', '')}\n\n{row.get('content', '')}"
    base_text = str(base_text).strip()
    if not base_text:
        continue

    chunks = splitter.split_text(base_text)
    for order, chunk in enumerate(chunks):
        rows.append({
            "id": idx,
            "case_title": row.get('case_title') or "",
            "case_no": row.get('case_no') or "",
            "court": row.get('court') or "",
            "decided_at": row.get('decided_at'),
            "chunk_ord": order,
            "content": chunk
        })
if not rows:
    raise RuntimeError("임베딩할 텍스트가 없습니다.")

# --------------------
# 임베딩 함수
# --------------------
def embed_batch(text_batch):
    resp = client.embeddings.create(model=EMB_MODEL, input=text_batch)
    return [e.embedding for e in resp.data]

# --------------------
# 배치 임베딩
# --------------------
BATCH = 128
embeddings = []
for i in range(0, len(rows), BATCH):
    batch_texts = [r["content"] for r in rows[i:i+BATCH]]
    emb = embed_batch(batch_texts)
    embeddings.extend(emb)

assert len(embeddings) == len(rows)

# rows에 임베딩 병합
for r, e in zip(rows, embeddings):
    r["embedding"] = e
print(rows[:10])


[{'id': 0, 'case_title': '특정범죄 가중처벌 등에 관한 법률 제6조 제7항 위헌제청', 'case_no': '2016헌가13', 'court': '헌법재판소', 'decided_at': datetime.date(2019, 2, 28), 'chunk_ord': 0, 'content': '특정범죄 가중처벌 등에 관한 법률 제6조 제7항 위헌제청', 'embedding': [0.00209327251650393, -0.014604690484702587, 0.016706276684999466, -0.030326679348945618, -0.02817188948392868, 0.023582983762025833, -0.0005665469216182828, -0.004828326404094696, -0.015934808179736137, 0.016267336905002594, 0.011445661075413227, -0.005782685708254576, -0.012230430729687214, -0.015043629333376884, -0.008925088681280613, 0.029608415439724922, 0.040169548243284225, -0.025032810866832733, 0.002109898952767253, -0.005120952147990465, 0.002804885385558009, 0.01066754199564457, -0.027666443958878517, 0.023050935938954353, 0.0043062553741037846, 0.006081962026655674, 0.013726812787353992, -0.040861207991838455, -0.004628808703273535, 0.016559962183237076, 0.010873710736632347, 0.0056230719201266766, -0.007255790755152702, -0.022066649049520493, -0.0072158873081

In [ ]:
#저장할 데이터 준비
insert_data = []
for chunk_info, embedding in zip(rows, embeddings):
    insert_data.append((
        chunk_info["case_title"],
        chunk_info["case_no"],
        chunk_info["court"], 
        chunk_info["decided_at"],
        chunk_info["chunk_ord"],
        chunk_info["content"],
        embedding
    ))

print(insert_data[:10])

16556


In [34]:
# --- DB upsert ---
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT")
)

cur = conn.cursor()

for (title,no,court,date,chunk_ord,content,emb) in insert_data:
    #params.append((title,no,court,date,content,emb))
    # 데이터베이스에 저장
    cur.execute(
        "INSERT INTO law.cases (case_title, case_no, court, decided_at, chunk_ord, content, embedding) VALUES (%s,%s,%s,%s,%s,%s,%s::vector)",
        (title,no,court,date,chunk_ord,content,emb)
    )

#execute_batch(cur, sql, params, page_size=200)
conn.commit()
cur.close()
conn.close()
print("ingest done")


ingest done


In [ ]:
# Langchain
import os
import psycopg2
from dotenv import load_dotenv
from typing import List

from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
load_dotenv()

EMB_MODEL = "text-embedding-ada-002"  # 기존 코드와 동일
EMB_DIM = "1536"

emb = OpenAIEmbeddings(model=EMB_MODEL)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    port=os.getenv("DB_PORT")
)

# --- SQL 기반 리트리버 ---
def _search_db(query: str, k: int = 5) -> List[Document]:
    qvec = emb.embed_query(query)  # list[float], 길이 EMB_DIM
    sql = """
    SELECT case_title, case_no, court, decided_at, chunk_ord, content,
           1 - (embedding <=> %s::vector) AS score
    FROM law.cases
    ORDER BY embedding <=> %s::vector
    LIMIT %s;
    """
    # 핵심: 파라미터를 그대로 넘기고 ::vector 캐스트로 해결
    cur = conn.cursor()
    cur.execute(sql, (qvec, qvec, k))
    rows = cur.fetchall()
    
    conn.commit()
    cur.close()
    conn.close()

    
    docs = []
    for (title, no, court, decided_at, ord_, content, score) in rows:
        meta = {
            "case_title": title,
            "case_no": no,
            "court": court,
            "decided_at": str(decided_at) if decided_at else None,
            "chunk_ord": ord_,
            "score": float(score),
        }
        docs.append(Document(page_content=content, metadata=meta))
    return docs

retriever = RunnableLambda(lambda q: _search_db(q["question"], k=q.get("k", 5)))

# --- 프롬프트 ---
SYSTEM = (
"너는 한국 저작권 판례 질의응답 보조자다. "
"주어진 컨텍스트 내에서만 근거를 들어 한국어로 답하라. "
"불확실하면 모른다고 말하라."
)
PROMPT = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("human",
     "질문:\n{question}\n\n"
     "검색 컨텍스트(상위 {k}개):\n"
     "{context}\n\n"
     "요구사항:\n"
     "1) 핵심 결론 먼저 한 줄\n"
     "2) 근거 문장 2~4개 인용(간단히)\n"
     "3) 관련 판례 메타데이터(case_no, court, decided_at) 명시\n"
     "4) 모르면 모른다고 답")
])

def _format_context(docs: List[Document]) -> str:
    out = []
    for d in docs:
        m = d.metadata
        head = f"[{m.get('case_no','?')} | {m.get('court','?')} | {m.get('decided_at','?')} | score={m.get('score'):.3f}]"
        out.append(head + "\n" + d.page_content.strip())
    return "\n\n---\n\n".join(out)

chain = (
    RunnableParallel(
        question=lambda x: x["question"],
        k=lambda x: x.get("k", 5),
        docs=retriever
    )
    | RunnableLambda(lambda x: {
        "question": x["question"],
        "k": x["k"],
        "context": _format_context(x["docs"])
    })
    | PROMPT
    | llm
)

# --- 간단 실행 함수 ---
def ask(question: str, k: int = 5):
    return chain.invoke({"question": question, "k": k}).content

q = "저작권 침해에 대한 손해배상액 산정 기준은?"
print(ask(q, k=5))


저작권 침해에 대한 손해배상액 산정 기준은 침해로 인한 실제 손해액 또는 침해자의 이익을 기준으로 산정된다.

대법원은 "저작권 침해로 인한 손해배상액은 침해로 인한 실제 손해액 또는 침해자가 얻은 이익을 기준으로 산정해야 한다"고 판시하였다. 또한, "손해배상액의 산정은 피해자의 입증 책임이 있으며, 이를 위해 필요한 자료를 제출해야 한다"고 언급하였다.

- 판례 메타데이터: 
  - case_no: 2009다80637
  - court: 대법원
  - decided_at: 2010-03-11


In [ ]:
# rag_retrievers.py
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.pgvector import PGVector
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.document_loaders import SQLDatabaseLoader
from langchain_community.utilities import SQLDatabase

load_dotenv()

# --- 공통 LLM + Embedding ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# --- DB 접속정보 ---
CONNECTION_STRING = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

# --- Vector Retriever ---
vectorstore = PGVector(
    connection_string=CONNECTION_STRING,
    embedding_function=embedding,
    collection_name="cases",     # law.cases 매핑됨
)
vector_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

# --- BM25 Retriever ---
# documents 전체를 불러오는 과정 필요 (ex: 초기화 시 한 번만 실행)
db = SQLDatabase.from_uri(CONNECTION_STRING, include_tables=["cases"], schema="law")

QUERY = """
SELECT content, case_no, case_title, court, decided_at
FROM law.cases
"""

# row는 dict로 들어옵니다.
def page_content_mapper(row):
    return row.get("content", "")

def metadata_mapper(row):
    return {
        "case_no": row.get("case_no"),
        "case_title": row.get("case_title"),
        "court": row.get("court"),
        "decided_at": row.get("decided_at"),
    }

loader = SQLDatabaseLoader(
    QUERY,
    db,
    page_content_mapper=page_content_mapper,
    metadata_mapper=metadata_mapper,
    source_columns=["case_no"]  # 선택: 메타데이터 source에 case_no 표시
)

docs = loader.load()

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 5

# --- Ensemble Retriever ---
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.6, 0.4]   # 임의 비율 (튜닝 가능)
)

c:\workspace\demo-app\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\SKAX\AppData\Local\Temp\ipykernel_53652\2215872716.py:25: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation.
  vectorstore = PGVector(
C:\Users\SKAX\AppData\Local\Temp\ipykernel_53652\2215872716.py:25: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering base

In [ ]:
# --- 공통 QA 체인 ---
PROMPT = ChatPromptTemplate.from_messages([
    ("system", "너는 저작권 판례 Q&A 어시스턴트다. 제공된 컨텍스트 내에서만 답하라."),
    ("human", "질문: {question}\n\n컨텍스트:\n{context}")
])

# --- Pre Retriever: HYDE - 가설 문서 생성 후 그걸로 검색 ---
HYDE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "너는 저작권 판례 Q&A 어시스턴트다."),
    ("human", "아래 질문에 대한 '가설 문서'를 5~7문장으로 한국어로 작성하라. "
              "사실처럼 자연스럽게 서술하되, 출처·문헌 언급은 금지.\n\n질문: {question}")
])

def hyde_expand(question: str) -> str:
    msgs = HYDE_PROMPT.format_messages(question=question)
    #print("===가상문서 생성(Pre Retriever: Hyde expand) ===")
    return llm.invoke(msgs).content.strip()


# --- Post Retriever ---
RERANK_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "문서와 질문의 관련성을 0-10점으로 평가하라. 숫자만 응답."),
    ("human", "질문: {question}\n\n문서: {document}\n\n관련성 점수 (0-10):")
])

# --- Post Retriever 1: Document Filtering - 중복/저품질 문서 제거 ---
def filter_documents(docs, min_length=50, similarity_threshold=0.8):
    #중복 및 저품질 문서 필터링
    #print("=== 문서 필터링(Post Retriever1: Document Filtering) ===")
    filtered_docs = []
    
    for doc in docs:
        # 1. 길이 필터링
        if len(doc.page_content.strip()) < min_length:
            #print(f"길이 부족으로 제외: {doc.page_content[:30]}...")
            continue
        
        # 2. 중복 필터링 (간단한 문자열 유사도)
        is_duplicate = False
        for existing_doc in filtered_docs:
            # 첫 100자 기준으로 중복 체크
            if doc.page_content[:100] == existing_doc.page_content[:100]:
                #print(f"중복으로 제외: {doc.page_content[:30]}...")
                is_duplicate = True
                break
        
        if not is_duplicate:
            filtered_docs.append(doc)
            #print(f"포함: {doc.page_content[:50]}...")
    
    return filtered_docs
#--- Post Retriever 2: Rerank - LLM 기반 관련성 재평가 
def rerank_documents(question: str, docs, top_k=5):
    #LLM을 사용해 문서를 재순위화
    #print("=== 문서 재순위화(Post Retriever2: Rerank) ===")
    scored_docs = []
    
    for doc in docs:
        msgs = RERANK_PROMPT.format_messages(question=question, document=doc.page_content[:500])
        try:
            score = float(llm.invoke(msgs).content.strip())
            scored_docs.append((doc, score))
            #print(f"점수: {score:.1f} - {doc.page_content[:50]}...")
        except:
            scored_docs.append((doc, 0.0))  # 파싱 실패시 낮은 점수
    
    # 점수 기준 내림차순 정렬 후 top_k 반환
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in scored_docs[:top_k]]

# PreRetriever + Retriever + PostRetriever Run chain
def run_chain_hyde_rerank(retriever, question: str):
    if retriever == vector_retriever:
        msgs = PROMPT.format_messages(question=question, context="")
        return llm.invoke(msgs).content
    else:
        #Pre Retriever: HYDE / 질문 + 가설문서를 합쳐서 검색 신호 강화
        hypo = hyde_expand(question)
        tuned_query = f"{question}\n\n{hypo}"

        # Retriever: 초기 문서 검색 (더 많이 가져오기)
        docs = retriever.get_relevant_documents(tuned_query)
        #print(f"초기 검색된 문서 수: {len(docs)}")
        
        # Post Retriever 1: Document Filtering
        filtered_docs = filter_documents(docs, min_length=30)
        
        # Post Retriever 2: Rerank
        reranked_docs = rerank_documents(question, filtered_docs, top_k=3)
        #print(f"최종 선택된 문서 수: {len(reranked_docs)}")

        #최종 context 생성 및 답변
        context = "\n---\n".join([d.page_content[:300] for d in reranked_docs])
        msgs = PROMPT.format_messages(question=question, context=context)
        return llm.invoke(msgs).content


if __name__ == "__main__":
    q = "저작권 침해 손해배상액 산정 기준은?"
    print("=== VectorRetriever ===")
    print(run_chain_hyde_rerank(vector_retriever, q))
    print("=== BM25Retriever ===")
    print(run_chain_hyde_rerank(bm25_retriever, q))
    print("=== EnsembleRetriever ===")
    print(run_chain_hyde_rerank(ensemble_retriever, q))

#rerank 등 2가지 정도만 RAG 테크닉 적용해보기

=== BM25Retriever ===


C:\Users\SKAX\AppData\Local\Temp\ipykernel_53652\300359434.py:78: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(tuned_query)


저작권 침해 손해배상액 산정 기준은 저작권법 제126조에 따라 손해가 발생한 사실이 인정되나, 구체적인 손해액을 산정하기 어려운 경우에는 변론의 취지 및 증거조사의 결과를 참작하여 상당한 손해액을 인정할 수 있습니다. 이때, 손해액은 침해자가 저작물을 사용 허락을 받았더라면 지급했을 객관적으로 상당한 금액으로 산정됩니다. 만약 저작권자가 해당 저작물에 대해 사용계약을 체결하거나 사용료를 받은 적이 없다면, 업계에서 일반화된 사용료를 기준으로 삼을 수 있습니다.
=== EnsembleRetriever ===
저작권 침해 손해배상액 산정 기준은 여러 요소를 고려하여 결정됩니다. 일반적으로는 다음과 같은 기준이 적용됩니다:

1. **실제 손해**: 저작권자가 입은 실제 손해를 기준으로 산정합니다. 이는 저작물이 침해된 경우, 저작권자가 해당 저작물의 사용으로부터 얻을 수 있었던 수익을 포함합니다.

2. **침해자의 이익**: 침해자가 저작권 침해를 통해 얻은 이익을 기준으로 할 수도 있습니다. 이는 침해자가 저작물을 사용하여 얻은 수익을 포함합니다.

3. **법정 손해배상**: 저작권법 제125조 제2항에 따라 손해액 산정이 어려운 경우, 법원은 법정 손해배상을 통해 일정 금액을 정할 수 있습니다.

4. **업계 관행**: 원고가 주장하는 손해배상액이 업계에서 일반화된 이용료와 일치하는지 여부도 고려됩니다. 그러나 이와 관련된 증거가 부족할 경우, 손해배상액 인정이 어려울 수 있습니다.

5. **유사 판례**: 유사한 사건에서의 판결 내용을 참고하여 손해액을 산정하기도 합니다.

결론적으로, 저작권 침해 손해배상액은 피해자의 실제 손해, 침해자의 이익, 업계 관행, 법정 손해배상 규정 등을 종합적으로 고려하여 결정됩니다.


In [ ]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy
)

# 기존 코드에서 retriever들과 체인 함수들은 그대로 사용
# (VectorRetriever, BM25Retriever, EnsembleRetriever, run_chain_hyde_rerank 함수 등)

def create_evaluation_dataset():
    #평가용 질문-답변 데이터셋 생성
    eval_questions = [
        "저작권 침해 손해배상액 산정 기준은?",
        "공정이용의 판단기준은 무엇인가?", 
        "저작권 보호기간은 얼마나 되는가?",
        "저작인격권과 저작재산권의 차이점은?",
        "온라인상 저작권 침해 대응방법은?"
    ]
    
    # 실제 사용 시에는 전문가가 작성한 정답을 사용해야 함
    ground_truths = [
        "저작권 침해 손해배상액은 침해로 인한 손해액, 침해자의 얻은 이익, 또는 사용료 상당액 중 하나를 기준으로 산정한다.",
        "공정이용은 이용목적, 저작물의 성질, 이용분량과 실질성, 시장효과 등을 종합적으로 고려하여 판단한다.",
        "저작권 보호기간은 원칙적으로 저작자 생존기간과 사후 70년간이다.",
        "저작인격권은 공표권, 성명표시권, 동일성유지권으로 구성되며 양도불가능하고, 저작재산권은 재산적 권리로 양도가능하다.",
        "온라인 저작권 침해시 온라인서비스제공자에게 삭제요청, 법정손해배상 청구, 형사고발 등의 방법이 있다."
    ]
    
    return eval_questions, ground_truths

def evaluate_retriever_with_ragas(retriever, retriever_name, questions, ground_truths):
    #특정 retriever에 대해 RAGAS 평가 수행
    print(f"\n=== {retriever_name} RAGAS 평가 시작 ===")
    
    # 각 질문에 대해 답변과 컨텍스트 생성
    answers = []
    contexts = []
    
    for question in questions:
        print(f"\n질문 처리중: {question}")
        
        # HyDE + Rerank 체인 실행하면서 중간 결과도 수집
        hypo = hyde_expand(question)
        tuned_query = f"{question}\n\n{hypo}"
        
        # 검색된 문서들
        docs = retriever.get_relevant_documents(tuned_query)
        filtered_docs = filter_documents(docs, min_length=30)
        reranked_docs = rerank_documents(question, filtered_docs, top_k=3)
        
        # 컨텍스트 생성
        context_list = [d.page_content[:300] for d in reranked_docs]
        contexts.append(context_list)
        
        # 최종 답변 생성
        context_str = "\n---\n".join(context_list)
        msgs = PROMPT.format_messages(question=question, context=context_str)
        answer = llm.invoke(msgs).content
        answers.append(answer)
    
    # RAGAS 평가용 데이터셋 구성
    data = {
        'question': questions,
        'answer': answers,
        'contexts': contexts,
        'ground_truths': ground_truths
    }
    
    dataset = Dataset.from_dict(data)
    
    # RAGAS 평가 메트릭 선택 (reference 컬럼 없이 사용 가능한 메트릭만)
    metrics = [
        faithfulness,          # 답변이 제공된 컨텍스트에 얼마나 충실한가
        answer_relevancy,      # 답변이 질문과 얼마나 관련있는가  
    ]
    
    # 평가 실행
    print("RAGAS 평가 실행중...")
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm,  # 평가에 사용할 LLM
        embeddings=embedding,  # 평가에 사용할 임베딩 모델
    )
    
    return result

def compare_retrievers():
    #retriever 성능 비교
    questions, ground_truths = create_evaluation_dataset()
    
    retrievers = [
        (bm25_retriever, "BM25Retriever"), 
        (ensemble_retriever, "EnsembleRetriever")
    ]
    
    results = {}
    
    for retriever, name in retrievers:
        try:
            result = evaluate_retriever_with_ragas(retriever, name, questions, ground_truths)
            results[name] = result.scores
            print(f"\n{name} 평가 완료:")
            
            for score in result.scores:
                for metric, score in score.items():
                    print(f" {metric}: {score:.4f}")
        except Exception as e:
            print(f"{name} 평가 중 오류 발생: {e}")
            results[name] = None
    
    return results

def print_comparison_report(results):
    #비교 결과 출력
    print("\n" + "="*50)
    print("RETRIEVER 성능 비교 리포트")
    print("="*50)
    
    # 사용 가능한 메트릭 동적 결정 (reference 불필요한 메트릭만)
    base_metrics = ['faithfulness', 'answer_relevancy']
    metrics = base_metrics.copy()
    
    df_data = []
    
    for retriever_name, result in results.items():
        if result is not None:
            for rst in result:
                row = [retriever_name]
                for metric in metrics:
                    score = rst.get(metric, 0.0)
                    row.append(f"{score:.4f}")
                df_data.append(row)
    
    df = pd.DataFrame(df_data, columns=['Retriever'] + metrics)
    print(df.to_string(index=False))
    
    # 각 메트릭별 최고 성능 retriever 찾기
    print("\n" + "-"*30)
    print("메트릭별 최고 성능:")
    print("-"*30)
    #results: metric별 성능 측정 결과 리스트.
    for metric in metrics:
        best_score = 0
        best_retriever = ""
        for retriever_name, result in results.items():
            if result is not None:
                #result list 중 하나씩 추출하여 best score 계산하도록 계산
                for rst in result:
                    score = rst.get(metric, 0.0)
                    if score > best_score:
                        best_score = score
                        best_retriever = retriever_name
        print(f"{metric}: {best_retriever} ({best_score:.4f})")

if __name__ == "__main__":
    print("RAGAS를 이용한 Retriever 성능 평가 시작")
    # 전체 비교 평가
    results = compare_retrievers()
    print_comparison_report(results)
    
    print("\n평가 완료!")

RAGAS를 이용한 Retriever 성능 평가 시작

=== BM25Retriever RAGAS 평가 시작 ===

질문 처리중: 저작권 침해 손해배상액 산정 기준은?

질문 처리중: 공정이용의 판단기준은 무엇인가?

질문 처리중: 저작권 보호기간은 얼마나 되는가?

질문 처리중: 저작인격권과 저작재산권의 차이점은?

질문 처리중: 온라인상 저작권 침해 대응방법은?
RAGAS 평가 실행중...


Evaluating: 100%|██████████| 10/10 [00:44<00:00,  4.44s/it]



BM25Retriever 평가 완료:
 faithfulness: 1.0000
 answer_relevancy: 0.8356
 faithfulness: 1.0000
 answer_relevancy: 0.8105
 faithfulness: 1.0000
 answer_relevancy: 0.8485
 faithfulness: 0.6154
 answer_relevancy: 0.8276
 faithfulness: 0.4545
 answer_relevancy: 0.8630

=== EnsembleRetriever RAGAS 평가 시작 ===

질문 처리중: 저작권 침해 손해배상액 산정 기준은?

질문 처리중: 공정이용의 판단기준은 무엇인가?

질문 처리중: 저작권 보호기간은 얼마나 되는가?

질문 처리중: 저작인격권과 저작재산권의 차이점은?

질문 처리중: 온라인상 저작권 침해 대응방법은?
RAGAS 평가 실행중...


Evaluating: 100%|██████████| 10/10 [00:51<00:00,  5.11s/it]



EnsembleRetriever 평가 완료:
 faithfulness: 1.0000
 answer_relevancy: 0.8272
 faithfulness: 1.0000
 answer_relevancy: 0.8106
 faithfulness: 0.5000
 answer_relevancy: 0.8234
 faithfulness: 0.8000
 answer_relevancy: 0.8278
 faithfulness: 1.0000
 answer_relevancy: 0.8643

RETRIEVER 성능 비교 리포트
        Retriever faithfulness answer_relevancy
    BM25Retriever       1.0000           0.8356
    BM25Retriever       1.0000           0.8105
    BM25Retriever       1.0000           0.8485
    BM25Retriever       0.6154           0.8276
    BM25Retriever       0.4545           0.8630
EnsembleRetriever       1.0000           0.8272
EnsembleRetriever       1.0000           0.8106
EnsembleRetriever       0.5000           0.8234
EnsembleRetriever       0.8000           0.8278
EnsembleRetriever       1.0000           0.8643

------------------------------
메트릭별 최고 성능:
------------------------------
faithfulness: BM25Retriever (1.0000)
answer_relevancy: EnsembleRetriever (0.8643)

평가 완료!
